# COVID-19 and Census Data 
___

<b> Table of Contents: </b>
<br> [0. Importing Libraries and Loading the Data](#0)
<br> [1. COVID Confirmed Cases as Target Variable](#1)
<br> [2. COVID Deaths as Target Variable](#2)

This dataset is made up of COVID-19 data tracking for each county in the tristate area, New York, New Jersey, and Connecticut. The csv file has data for COVID-19 cases and deaths on every single day since Janurary 22, 2020 through June 10, 2020. This project utlizes this data source because of the relability of its' county level data. The Census Economic and Income data is made up of county level data and in order to combine the two datasets there needed to be consistency in the formats. 

Data source: USAFACTS https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/
- COVID-19 Confirmed Cases and Deaths
- Population (population data from 2019 US county Data)

Data range: 
- Janurary 22, 2020 - June 9, 2020 (stopped gathering 2 weeks before final Capstone was due to leave time for data cleaning and prep)


Quick Notes on Data Cleaning:
- Calculated total cases and deaths per month
- Changed state abbreviations to full name to match the Census data
- Fill in state name for any counties that were left blank 
- Compare the county names and order from this dataset to the Census data 
- Delete counties not present in both datasets
- Add county population counts if listed as 0

<a id = "0"> <h2> 0. Importing Libraries and Loading the Data </h2> </a>
___

_Importing Libraries_

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
import numpy as np
import csv

_Loading the Data_

In [2]:
df_tristate_cases = pd.read_csv("tristate_final_data.csv")
df_tristate_cases.head()

,NAME,Covid Confirmed Cases,Covid Deaths,Population,Covid Case Rate (per 1000),Covid Death Rate (per 1000),Households SNAP,Estimated Individuals SNAP,SNAP % Population,SNAP Per Capita Benefit or TAM,...,Other Race Alone,Hispanic or Latino,Median Age,Male Median Age,Female Median Age,Total Households,Average Household Size,Total Families,state code,county code
0,"Albany County, New York",87943,3989,304204,289.092188,13.112911,"15,087","34,247",11%,"$51,473,977.00",...,"7,647","289,287",38.5,36.8,40.0,"126,251",2,"60,631",36,1
1,"Allegany County, New York",2601,126,48946,53.140195,2.574266,"2,994","7,305",15%,"$10,979,956.00",...,557,"48,276",37.8,36.2,39.2,"18,208",2,"10,576",36,3
2,"Bronx County, New York",2544478,234021,1385108,1837.024983,168.955056,"184,934","512,267",37%,"$769,937,572.00",...,"73,243","643,695",32.8,30.6,34.9,"483,449",3,"368,196",36,5
3,"Broome County, New York",24218,1802,200600,120.727817,8.983051,"13,226","30,684",15%,"$46,118,533.00",...,"5,087","193,822",40.2,38.1,42.2,"82,167",2,"40,559",36,7
4,"Cattaraugus County, New York",3875,128,80317,48.246324,1.593685,"5,801","13,980",17%,"$21,012,556.00",...,"1,363","78,972",40.7,39.8,41.6,"32,263",2,"18,801",36,9


<b> Number of rows, number of columns in dataset </b>

In [8]:
print('There are # number of rows in the dataset    :', df_tristate_cases.shape[0])
print('There are # number of columns in the dataset :', df_tristate_cases.shape[1])

There are # number of rows in the dataset    : 91
There are # number of columns in the dataset : 43


In [9]:
df_tristate_cases.describe().round(2)

,Covid Confirmed Cases,Covid Deaths,Population,Covid Case Rate (per 1000),Covid Death Rate (per 1000),Median Age,Male Median Age,Female Median Age,Average Household Size,state code,county code
count,91.00,91.00,91.00,91.00,91.00,91.00,91.00,91.00,91.00,91.00,91.00
mean,345605.69,24234.99,348836.19,494.78,30.01,40.16,38.84,41.45,2.44,33.16,47.79
std,706010.22,59348.61,468866.99,569.68,36.52,3.10,3.23,3.02,0.50,7.59,36.96
min,304.00,0.00,4836.00,16.82,0.00,29.80,28.50,31.10,2.00,9.00,1.00
25%,5696.00,222.50,64956.00,88.36,3.93,38.50,37.00,39.90,2.00,34.00,15.00
50%,36536.00,1815.00,149265.00,230.12,13.46,40.40,39.20,41.90,2.00,36.00,37.00
75%,297332.00,19315.00,467878.00,698.36,48.04,41.70,40.55,42.95,3.00,36.00,78.00
max,3587059.00,347696.00,2504700.00,2523.79,168.96,51.30,50.90,51.80,3.00,36.00,123.00


In [10]:
df_tristate_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 43 columns):
NAME                                          91 non-null object
Covid Confirmed Cases                         91 non-null int64
Covid Deaths                                  91 non-null int64
Population                                    91 non-null int64
Covid Case Rate (per 1000)                    91 non-null float64
Covid Death Rate (per 1000)                   91 non-null float64
 Households SNAP                              91 non-null object
 Estimated Individuals SNAP                   91 non-null object
 SNAP % Population                            91 non-null object
 SNAP Per Capita Benefit or TAM               91 non-null object
 Total Citizen Educated in US                 91 non-null object
 Citizen Less than High School  Education     91 non-null object
 Citizen High School  Graduate                91 non-null object
 Citizen Some College  Education              91 non-